In [ ]:
import pandas as pd
from bandata.api import get
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [ ]:
# month value to use when calculating months since September
sept = 9

## Part 1: Querying the Data

In [ ]:
# Query Redshift for the JD Power Valuations
import psycopg2
conn = psycopg2.connect(
    host='xxxx',
    database='xxx',
    port=xxx,
    user='xx',
    password='xx'
)

# Query a table using the Cursor
sql = """
WITH all_data AS
(
    SELECT
        mv.bam_period as period,
        mv.bam_region AS region_id,
        r.region_name,
        mv.bam_ucgvehicleid as ucgvehicleid,
        vl.YEAR,
        vl.make,
        vl.model,
        vl.bodystyle,
        mv.basecleantrade,
        mv.basecleanretail,
        mv.averagemileage,
        ma.lowlevel,
        ma.highlevel,
        ma.adjamount,
        mv.baseaverageetrade + ma.adjamount AS adjustedcleantrade,
        mv.basecleanretail + ma.adjamount AS adjustedcleanretail
    FROM bigdataprod.jdpower.v_monthly_valuations_latest_by_date AS mv
    JOIN bigdataprod.jdpower.v_vehicle_list_latest_by_date AS vl
    ON mv.bam_ucgvehicleid = vl.ucgvehicleid
    JOIN jdpower.regions AS r
    ON mv.bam_region = r.region_id
    JOIN jdpower.v_mileage_adjustments_latest_by_date AS ma
    ON mv.bam_ucgvehicleid = ma.bam_ucgvehicleid
    AND mv.bam_period = ma.bam_period
)

SELECT
    ucgvehicleid, -- The JD Power Vehicle ID that identifies a Make/Model/Year/Trim
    period, -- The period that the valuation is for
    region_id, -- Region ID that the valuation is for
    region_name, -- Name of the region
    make, -- Make of vehicle
    model, -- Model of vehicle
    YEAR, -- Vehicle model year
    bodystyle, -- Essentially the trim level
    basecleantrade, -- Trade in value given the average mileage for this model year in this period
    basecleanretail, -- Retail value given the average mileage for this model year in this period
    averagemileage, -- Average mileage for this model and year in the selected period
    lowlevel, -- Low mileage for this model and year in the selected period
    highlevel, -- High mileage for this model and year in the selected period
    adjamount, -- Adjustment amount for an instance of this ucgvehicleid
    adjustedcleantrade, -- Clean trade value for a 30K version of this ucgvehicleid with 30,000 miles
    adjustedcleanretail -- Clean retail value for a 30K version of this ucgvehicleid
FROM all_data
WHERE
    (30000 BETWEEN lowlevel AND highlevel) -- update mileage anchor as desired
    AND basecleanretail <> 0
ORDER BY
    ucgvehicleid,
    period,
    region_id
"""

# Uncomment
dfResults = pd.read_sql(sql, conn)
conn.close()

In [ ]:
# lets work on a copy of the data so we don't need to requery if we want to start again.
dfResultsWithName = dfResults.copy()

In [ ]:
# Pull actual CPI data for autos
# Convert the period to a date
dfResultsWithName['period'] = pd.to_datetime(dfResultsWithName['period']).dt.date

# get the min and max dates out of the valuation results data
mindate = datetime.datetime.combine(dfResultsWithName['period'].min(), datetime.datetime.min.time())
maxdate = datetime.datetime.combine(dfResultsWithName['period'].max(), datetime.datetime.min.time())

# get CPI Data for this range
# CPRTUCTR Index - US CPI Urban Consumers Used Cars & Trucks NSA
# CPSTUCTR Index - US CPI Urban Consumers Used Cars & Trucks SA

dfCPI = get(
    "BloombergApi.BloombergHistoricalData",
    Auth=dict(Type='App', Name='BAM:SDA'),
    api_parameters=dict(data_environment='PROD'),  # PROD for production
    Symbols=dict(Symbolology='Ticker', IDs=('CPRTUCTR Index')),
    Fields=('PX_OPEN', 'PX_LAST', 'PX_VOLUME'),
    Periodicity='MONTHLY',
    StartDate=pd.Timestamp(mindate),
    EndDate=pd.Timestamp(maxdate)
)

# make sure period columns is a date
dfCPI['period'] = pd.to_datetime(dfCPI['TIMESTAMP']).dt.date

# cosmetic change to column name
dfCPI.rename(columns={'PX_LAST': 'CPRTUCTR'}, inplace=True)
dfCPI.drop(['SYMBOL', 'TIMESTAMP'], axis=1, inplace=True)

# Calculate month over month change in inflation
dfCPI['pctChangeInflation'] = dfCPI['CPRTUCTR'].pct_change()

dfCPI.head()

In [ ]:
# merge CPI data into valuation data
dfResultsWithName = pd.merge(dfResultsWithName, dfCPI, how='left', on='period')
dfResultsWithName.head()

In [ ]:
# Create analysis dataframe with a subset of the columns
dfAnalysis = dfResultsWithName[['ucgvehicleid', 'region_id', 'region_name', 'period', 'basecleanretail', 'averagemileage', 
                                'adjustedcleanretail', 'year', 'make', 'model', 'CPRTUCTR', 'pctChangeInflation']].copy()

# make sure retail number is numeric
dfAnalysis['adjustedcleanretail'] = pd.to_numeric(dfAnalysis['adjustedcleanretail'])

# filter just for testing. todo: remove
dfAnalysis = dfAnalysis[(dfAnalysis['ucgvehicleid']==20150805) & (dfAnalysis['region_id']==1)].head(10)

In [ ]:
# Calculate month over month change in price
dfAnalysis = dfAnalysis.sort_values(['ucgvehicleid', 'region_id', 'period'])
dfAnalysis['PctChangeAdjustedCleanRetail'] = dfAnalysis.groupby(['ucgvehicleid', 'region_id'])['adjustedcleanretail'].pct_change()
dfAnalysis.head()

In [ ]:
# Create a datetime column so that we can use the dt.month function
dfAnalysis['periodDT'] = pd.to_datetime(dfAnalysis['period'], errors='coerce', utc=True)

# Calculate months since September
dfAnalysis['months_since_september'] = (dfAnalysis['periodDT'].dt.month - sept) % 12

# Drop the date time column we don't need anymore
dfAnalysis.drop(['periodDT'], axis=1, inplace=True)

#### Repetition of above steps using local csv (for personal use)

In [ ]:
# Path to local csv
jdp = pd.read_csv('jdp.csv')  # dfResultsWithName from above saved locally as a csv file
jdp['period'] = pd.to_datetime(jdp['period'])

# Read CPI data from a local csv file
dfCPI = pd.read_csv('cpi.csv')
dfCPI['period'] = pd.to_datetime(dfCPI['period'])

# Merge CPI data into valuation data
jdp = pd.merge(jdp, dfCPI, how='left', on='period')

# Create analysis dataframe with a subset of the columns
dfAnalysis = jdp[['ucgvehicleid', 'region_id', 'region_name', 'period', 'basecleanretail', 
                  'averagemileage', 'adjustedcleanretail', 'year', 'make', 'model', 
                  'CPRTUCTR', 'PctChangeInflation']].copy()

# Make sure retail number is numeric
dfAnalysis['adjustedcleanretail'] = pd.to_numeric(dfAnalysis['adjustedcleanretail'])

# Calculate month over month change in price
dfAnalysis = dfAnalysis.sort_values(['ucgvehicleid', 'region_id', 'period'])
dfAnalysis['PctChangeAdjustedCleanRetail'] = dfAnalysis.groupby(['ucgvehicleid', 'region_id'])['adjustedcleanretail'].pct_change()

# Create a datetime column so that we can use the dt.month function
dfAnalysis['periodDT'] = pd.to_datetime(dfAnalysis['period'], errors='coerce', utc=True)

# Calculate months since September
dfAnalysis['months_since_september'] = (dfAnalysis['periodDT'].dt.month - sept) % 12

# Drop the date time column we don't need anymore
dfAnalysis.drop(['periodDT'], axis=1, inplace=True)

# Convert 'period' back to datetime after dropping 'periodDT'
dfAnalysis['period'] = pd.to_datetime(dfAnalysis['period'])

# Display the first few rows of the analysis dataframe
dfAnalysis.head()


## Part 2: Manipulating the data

#### Calculating depreciation adjusted prices

In [ ]:
# new df named "DAP" for Depreciation Adjusted Price
dap = dfAnalysis.copy()
dap = dap.sort_values(['ucgvehicleid', 'region_id', 'period'])

# Get adjusted clean retail price for next vintage (v+1)
vp1 = dap[['ucgvehicleid', 'period', 'make', 'model', 'year', 'region_id', 'adjustedcleanretail']].copy()
vp1['year'] = vp1['year'] - 1
vp1 = vp1.rename(columns={'ucgvehicleid': 'ucgvehicleid_v', 'adjustedcleanretail': 'adjustedcleanretail_vp1'})

# Merge to calculate depreciation adjusted price
dap = pd.merge(dap, vp1, on=['period', 'make', 'model', 'year', 'region_id'])
dap['depreciation_adj_price'] = dap['adjustedcleanretail'] * (dap['adjustedcleanretail_vp1'] / dap['adjustedcleanretail'] + 1) ** (dap['months_since_september'] / 12)

# Display the first few rows of the DAP dataframe
dap.head()

In [ ]:
# Get percent changes in adjusted clean retail (ACR) prices and depreciation adjusted prices (DAP)
dap['pctDAP'] = dap.groupby(['ucgvehicleid', 'region_id'])['depreciation_adj_price'].pct_change()
dap['pctACR'] = dap.groupby(['ucgvehicleid', 'region_id'])['adjustedcleanretail'].pct_change()

# Get average values for each period
dap['avg_pctDAP'] = dap.groupby('period')['pctDAP'].transform('mean')
dap['avg_pctACR'] = dap.groupby('period')['pctACR'].transform('mean')

#### Adding "smoothing" to avoid September drops without model changeover

In [ ]:
# months_since_september=12 in September rather than 0
dap['months_since_september2'] = dap['months_since_september'].replace(0, 12)

# quality adjustment set to 1
quality_adjustment = 1

# depreciation adjusted price calculation using altered months_since_september
dap['depreciation_adj_price2'] = dap['adjustedcleanretail'] * ((dap['adjustedcleanretail_vp1'] / dap['adjustedcleanretail'] + 1) ** (dap['months_since_september2'] / 12))

# Percent change in depreciation adjusted prices using altered months_since_september
dap['pctDAP2'] = dap.groupby(['ucgvehicleid', 'region_id'])['depreciation_adj_price2'].pct_change()

# Get average values for depreciation adjusted prices percent change
dap['avg_pctDAP2'] = dap.groupby('period')['pctDAP2'].transform('mean')
dap['avg_pctDAP_smooth'] = dap['avg_pctDAP']

# Replace percent change in September with new value; avoids issue where months_since_september changes from 11 to 0
dap.loc[dap['period'].dt.month == sept, 'avg_pctDAP_smooth'] = dap['avg_pctDAP2']

# Display the first few rows to verify changes
dap.head()

#### Model Changeover

In [ ]:
# Get vintage year associated with a particular period and specified age value
def get_vintage(date, age) -> int:
    date = pd.to_datetime(date)
    if date.month < sept:  # Assuming 'sept' is previously defined as 9
        # Get older vintage if month is before September
        return date.year - (age + 1)
    else:
        # Get 1 year newer vintage otherwise
        return date.year - age
   
for i in range(2, 7):
    # Create new dataframe for each age value 2-6
    dfVi = dap.copy()
    # Limit dataframe to vehicles of the specified age
    dfVi = dfVi[dfVi['year'] == dfVi['period'].apply(lambda x: get_vintage(x, i))]
    # Get pct change in DAP grouped by make, model, region ID, and age; we don't group by ID since ID/year changes every September
    dfVi['pctDAP_changeover'] = dfVi.groupby(['make', 'model', 'region_id'])['depreciation_adj_price'].pct_change()
    
    dfVi['age'] = i
    if i == 2:
        dfV = dfVi
    else:
        dfV = pd.concat([dfV, dfVi])

col_names = []
# Get average pct DAP for each individual age value 2-6
for i in range(2, 7):
    dfV['avg_pctDAP_changeover' + str(i) + 'yr'] = dfV.groupby('period')['pctDAP_changeover'].transform(lambda x: x[dfV['age'] == i].mean())
    col_names.append('avg_pctDAP_changeover' + str(i) + 'yr')

# Get average pct DAP across all age values 2-6
dfV['avg_pctDAP_changeover'] = dfV.groupby('period')['pctDAP_changeover'].transform('mean')

dfV.head()

In [ ]:
# Isolate columns of interest from model changeover dataframe (dfV)
dfV_columns = ['period', 'ucgvehicleid', 'region_id', 'pctDAP_changeover', 'avg_pctDAP_changeover'] + col_names

# Add model changeover calculations to original dataframe
dap = pd.merge(dap, dfV[dfV_columns], on=['period', 'ucgvehicleid', 'region_id'], how='left')  # Left merge such that all dates remain

In [ ]:
# Date selected such that all columns are populated
dap[dap['period'] == '2021-11-30'].head()

#### Calculating Regional Weights

In [ ]:
import requests
from census import Census
from us import states

# Uncomment as needed

# Get corresponding J.D. Power region ID for each state. Does not decrement license.
def get_request(endpoint:str, arguments:str, prod=False, echoRequestString = False):
    # API keys - pull from vault eventually
    prodAPIKey = "d8fa95b9-987b-4538-aa96-b5ce684bf2c2"
    TestAPIKey = "4d7cc8ce-1cbc-49ce-a918-840a861a1825"

    # URLs - Pull from consul eventually. Dev URL affects license count and has the same data as prod, but will have new features and end points a few weeks before prod url
    prodURL = "https://cloud.jdpower.ai/data-api/valuationservices/valuation/"
    devURL = "https://cloud.jdpower.ai/data-api/UAT/valuationservices/valuation/"

    # Create empty dataframe for results
    dfResult = pd.DataFrame()
    # Set API key and URL based on whether we want prod or test
    if prod == True:
        requestString = prodURL + endpoint + arguments
        apiKey = prodAPIKey
    else:
        requestString = devURL + endpoint + arguments
        apiKey = TestAPIKey

    # The request string is nice to have when talking to JD Power support
    if echoRequestString == True:
        print(requestString)

    # Make the request
    r = requests.get(requestString, headers={"api-key": apiKey})
    if r.status_code == 200:
        if echoRequestString == True:
            print('Success: ' + requestString)

        # Store resulting json
        json = r.json()

        # Convert results to a dataframe from json['result']
        dfResult = pd.DataFrame(json['result'])

    # else clause for failed HTTP request
    else:
        print('Failure')
        print(r)

    # Return result dataframe. Will be empty if the request failed
    return dfResult

# Variables for API endpoint and parameters
endpoint = 'regionByIDStateCode'
period = '2022-09-30'
userinfo = 'xxxx'

# Codes represent different U.S. states
codes = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 
            'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY', 'DC']

# Initialize an empty DataFrame for collecting regions
dfRegions = pd.DataFrame()


for i in range(len(codes)):
    arguments = '?statecode=' + codes[i] + '&userinfo=' + userinfo
    dfState = get_request(endpoint=endpoint, arguments=arguments, prod=True, echoRequestString=False)
    dfState.insert(0, 'state', codes[i])
    dfRegions = pd.concat([dfRegions, dfState])

# Change to personal Census API Key
c = Census("xxxx")

# Define list of states
state_list = [state.name for state in states.STATES]

# Retrieve population data for each state
populations = {}
for state in state_list:
    # Fetch population data using Census API, querying for state FIPS codes
    population_data = c.sf1.get(("NAME", 'P001001'), {'for': 'state:{}'.format(states.lookup(state).fips)})
    populations[state] = population_data[0]['P001001']

# Add District of Columbia
population_data = c.sf1.get(("NAME", 'P001001'), {'for': 'state:{}'.format('11')})
populations['District of Columbia'] = population_data[0]['P001001']


us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI"
}

# Maps state abbreviations to full state names
us_abbrev_to_state = {v: k for k, v in us_state_to_abbrev.items()}

# Create a copy of the dfRegions dataframe
dfPop = dfRegions.copy()

# Get 2010 state population given state abbreviation
def get_pop(abbrev):
    state = us_abbrev_to_state[abbrev]
    return populations[state]

# Add population data to dfPop
dfPop['pop'] = dfPop['state'].transform(lambda x: get_pop(x))
dfPop['pop'] = dfPop['pop'].astype(int)

# Get total population for each region
dfRegional = dfPop[['regionid', 'pop']].groupby('regionid').sum()

# Calculate weights as a percentage of total population
dfRegional['pop'] = dfRegional['pop'] / dfRegional['pop'].sum()
dfRegional = dfRegional.sort_values(by='pop', ascending=False)
dfRegional = dfRegional.reset_index()

dfRegional = dfRegional.rename(columns={'regionid': 'region_id'})
dfRegional['region_id'] = dfRegional['region_id'].astype(int)
dap['region_id'] = dap['region_id'].astype(int)
dap = pd.merge(dap, dfRegional, on='region_id')
dap = dap.rename(columns={'pop': 'region_weight'})

In [ ]:
# For personal use - storing regional weights so I don't need to rerun the previous cell
dfRegional.to_csv('region_weights.csv', index=False)

# For personal use - reading regional weights from locally saved CSV
dfRegional = pd.read_csv('region_weights.csv')
dap['region_id'] = dap['region_id'].astype(int)
dap = pd.merge(dap, dfRegional, on='region_id')
dap = dap.rename(columns={'pop': 'region_weight'})

# Display the DataFrame
dfRegional

In [ ]:
# Value we want to average using regional weights
value = 'pctDAP'

# Take the average of the specified value within each region for each period
by_region = dap.groupby(['region_id', 'period']).agg({value: 'mean', 'region_weight': 'first'})
by_region = by_region.reset_index()

# Use regional weights to compute a weighted average for each period
weighted_avg = by_region.groupby('period').apply(
    lambda x: (x[value] * x['region_weight']).sum()
)
weighted_avg = pd.DataFrame(weighted_avg)
weighted_avg = weighted_avg.rename(columns={0: 'avg_' + value + '_weighted'})

# Add weighted average to main df
dap = dap.merge(weighted_avg, on='period')

## Part 3: Graphing

In [ ]:
import matplotlib.pyplot as plt
import datetime as dt

# Define the estimates and titles
estimates = ['avg_pctACR', 'avg_pctDAP', 'avg_pctDAP_smooth', 'avg_pctDAP_changeover']  # 'avg_pctDAP_weighted'
titles = ['ACR Price', 'DAP', 'Smoothed DAP', 'DAP with Model Changeover']

# Loop through each estimate and create a subplot
for i in range(len(estimates)):
    estimate = estimates[i]
    title = titles[i]
    fig, ax = plt.subplots()

    # Add vertical lines for September 1st of each year
    # Comment to omit
    for year in range(2016, 2024):  # May need to alter dates according to estimation variable
        ax.axvline(dt.datetime(year, 9, 30), color='c', linestyle='--')

    dfGraph = dap[['period', 'pctChangeInflation', estimate]]
    dfGraph = dfGraph.dropna().drop_duplicates()
    dfGraph.plot(x='period', y=['pctChangeInflation', estimate], ax=ax)

    legend = plt.legend()
    legend.get_texts()[0].set_text('True Change')
    legend.get_texts()[1].set_text('Predicted Change')
    plt.title('Monthly UTC Predictions Using '+ title)  # Change title as appropriate
    plt.xlabel('Date')
    plt.ylabel('MoM Percent Change')
    plt.show()

## Part 4: Numerica Results

#### Correlation coefficients, R-squared values, and hit rates evaluating each variable as an estimate of inflation (UCT)


In [ ]:
estimates = ['avg_pctACR', 'avg_pctDAP', 'avg_pctDAP_smooth', 'avg_pctDAP_changeover', 'avg_pctDAP_weighted']

for estimate in estimates:
    dfClean = dap[['period', 'pctChangeInflation', estimate]]
    dfClean = dfClean.dropna().drop_duplicates()
    # Limit timeframe to 2018 onward due to sample size
    dfClean = dfClean[dfClean['period'] >= '2018-01-01']

    corr_coef = dfClean['pctChangeInflation'].corr(dfClean[estimate])
    r_squared = corr_coef ** 2
    hit_rate = ((dfClean['pctChangeInflation'] * dfClean[estimate]) >= 0).mean()
    print('Estimate: ' + estimate + ', Corr Coef: ' + str(corr_coef) + ', R-squared: ' + str(r_squared) + ', Hit Rate: ' + str(hit_rate))

Estimate: avg_pctACR
  Corr Coef: 0.85458410631952
  R-squared: 0.730270077178805
  Hit Rate: 0.7878787878787878

Estimate: avg_pctDAP
  Corr Coef: 0.9340578611863622
  R-squared: 0.872463715497617
  Hit Rate: 0.9090909090909091

Estimate: avg_pctDAP_smooth
  Corr Coef: 0.8653702544413996
  R-squared: 0.7495799593855096
  Hit Rate: 0.8787878787878788

Estimate: avg_pctDAP_changeover
  Corr Coef: 0.8555447720240712
  R-squared: 0.731965873073135
  Hit Rate: 0.06060606060606061

Estimate: avg_pctDAP_weighted
  Corr Coef: 0.934053712851616
  R-squared: 0.872463384918891
  Hit Rate: 0.9090909090909091


In [ ]:
print('Correlation coefficients by age value')

corr_coef = dap['pctChangeInflation'].corr(dap['avg_pctDAP_changeover'])
r_squared = corr_coef ** 2
print('All values 2-6:', corr_coef, r_squared)

for i in range(2, 7):
    corr_coef = dap['pctChangeInflation'].corr(dap['avg_pctDAP_changeover' + str(i) + 'yr'])
    r_squared = corr_coef ** 2
    print('Age value = ' + str(i) + ':', corr_coef, r_squared)

# Correlation over recent years
dfByAge = dap.copy()
dfByAge = dfByAge[dfByAge['period'] >= '2021-0{}-01'.format(sept)]
print('Correlation coefficients since Sep 2021')

corr_coef = dfByAge['pctChangeInflation'].corr(dfByAge['avg_pctDAP_changeover'])
r_squared = corr_coef ** 2
print('All values 2-6:', corr_coef, r_squared)

for i in range(2, 7):
    corr_coef = dfByAge['pctChangeInflation'].corr(dfByAge['avg_pctDAP_changeover' + str(i) + 'yr'])
    r_squared = corr_coef ** 2
    print('Age value = ' + str(i) + ':', corr_coef, r_squared)

dfByAge['avg_pctDAP_changeover34'] = (dfByAge['avg_pctDAP_changeover3yr'] + dfByAge['avg_pctDAP_changeover4yr']) / 2
corr_coef = dfByAge['pctChangeInflation'].corr(dfByAge['avg_pctDAP_changeover34'])
r_squared = corr_coef ** 2
print('Age values 3 & 4:', corr_coef, r_squared)

#### Classification reports and conclusion matrices for each estimate of inflation (UCT)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

estimates = ['avg_pctACR', 'avg_pctDAP', 'avg_pctDAP_smooth', 'avg_pctDAP_changeover', 'avg_pctDAP_weighted']

for estimate in estimates:
    dfClean = dap[['period', 'pctChangeInflation', estimate]]
    dfClean = dfClean.dropna().drop_duplicates()
    
    # Limit timeframe to 2018 onward due to sample size
    dfClean = dfClean[dfClean['period'] >= '2018-01-01']
    
    # Generate a confusion matrix and classification report for the current estimate
    y_true = dfClean['pctChangeInflation'] >= 0
    y_pred = dfClean[estimate] >= 0
    cm = confusion_matrix(y_true, y_pred)
    cr = classification_report(y_true, y_pred)
    
    # Print the results for the current estimate
    print(f'Results for {estimate}:')
    print('Classification Report:')
    print(cr)
    
    # Create heatmap of confusion matrix
    sns.heatmap(cm, annot=True, cmap='Blues', fmt='g')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix: ' + estimate)
    plt.show()


#### Changes in the sample through time

In [ ]:
# Unique vehicle ID's present in the sample over time
region_sample_counts = dap.groupby(['region_id', 'period'])['ucgvehicleid'].nunique().reset_index()
region_sample_counts.plot(x='period', y='ucgvehicleid', ylabel='vehicle count', title="Number of Unique Vehicle ID's by Period")
plt.show()

#### OLS Regression to combine the results

In [ ]:
import statsmodels.api as sm

def normalize(x):
    return (x - x.min()) / (x.max() - x.min())

cols = ['avg_pctACR', 'avg_pctDAP', 'avg_pctDAP_smooth', 'avg_pctDAP_changeover']

dfModel = dap[['period', 'pctChangeInflation'] + cols]
dfModel = dfModel[dfModel['period'] >= '2018-01-01']
dfModel = dfModel.dropna()
dfModel = dfModel.drop_duplicates()

dfModel[cols] = dfModel[cols].apply(normalize)

x = dfModel[cols]
y = dfModel['pctChangeInflation']

x = sm.add_constant(x)

model = sm.OLS(y, x).fit(cov_type='HC3')
print(model.summary())

In [ ]:
y_pred = model.predict(x)
dfGraph = dfModel

dfGraph['predicted_values'] = y_pred
dfGraph['true_values'] = y

dfGraph.plot(x='period', y=['predicted_values', 'true_values'], xlabel='Date', ylabel='MoM Percent Change', title='Monthly UCT Predictions Using OLS')

In [ ]:
corr_coef = dfGraph['true_values'].corr(dfGraph['predicted_values'])
r_squared = corr_coef ** 2
hit_rate = ((dfGraph['true_values'] >= 0) == (dfGraph['predicted_values'] >= 0)).mean()

print('Corr Coef: ' + str(corr_coef) + ', R-squared: ' + str(r_squared) + ', Hit Rate: ' + str(hit_rate))

In [ ]:
# Generate a confusion matrix and classification report for OLS
y_true_bool = y >= 0
y_pred_bool = y_pred >= 0
cm = confusion_matrix(y_true_bool, y_pred_bool)
cr = classification_report(y_true_bool, y_pred_bool)

# Print the results for the current estimate
print('Results for OLS:')
print('Classification Report:')
print(cr)

# Create heatmap of confusion matrix
sns.heatmap(cm, annot=True, cmap='Blues', fmt='g')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix: OLS')
plt.show()

#### July estimate of the UCT

In [ ]:
cols = ['avg_pctACR', 'avg_pctDAP', 'avg_pctDAP_smooth', 'avg_pctDAP_changeover']

dfModel = dap[['period', 'pctChangeInflation'] + cols]
dfModel = dfModel[dfModel['period'] >= '2018-01-01']
dfModel = dfModel.dropna()
dfModel = dfModel.drop_duplicates()

x = dfModel[cols]
y = dfModel['pctChangeInflation']

x = sm.add_constant(x)

model = sm.OLS(y, x).fit(cov_type='HC3')

x_last_row = dap[['period'] + cols].dropna().tail(1)
print(x_last_row)
x_last_row = x_last_row.drop('period', axis=1)
x_last_row.insert(0, 'const', 1)
x_last_row = x_last_row.reset_index(drop=True)

estimate = model.predict(x_last_row)
print(f'OLS Estimate: {estimate.iloc[0]:.6f}')

#### Plot percent change in ACR by region, will see little diff which explains why regional weighting makes little difference

In [ ]:
values = ['pctACR', 'adjustedcleanretail']
for value in values:
    region_data = dap.groupby(['region_id', 'period'])[value].mean()
    region_data = pd.DataFrame(region_data)
    region_data.reset_index(inplace=True)
    region_data['period'] = pd.to_datetime(region_data['period'])
    region_data = region_data.pivot(index='period', columns='region_id', values=value)
    
    
    region_data.plot(ylabel='average percent change in vehicle price', title=value + ' by region')

    region_data = region_data.reset_index()
    # Changes are clearer over a more limited window
    region_data = region_data[region_data['period'] >= '2018-01-01']
    region_data = region_data[region_data['period'] < '2019-01-01']
    
    ax = region_data.plot(x='period', xlabel='period', title=value + ' by region (2018)')
    
    # Add the horizontal line at y=0, useful for graphing % change
    # ax.axhline(y=0, color='gray', linestyle='--')

    plt.show()